# OPEN API

데이터를 수집한 곳에서 일반 개발자들에게 사용할 수 있도록 공개한 서비스.

유료/무료... 

대부분이 무료로 제공되고 사용량에 따라 요금을 받는다.

데이터는 XML이나 JSON 양식으로 제공하며 웹으로 요청이 가능하게 한다.

사용방법
1. OPEN API 서비스를 찾는다.
2. 요청할 주소를 확인한다.
3. 요청할 때 서버로 보낼 데이터(파라미터, 해더)를 확인한다.
4. 응답결과 양식을 확인한다.
5. postman으로 테스트.

In [16]:
import requests
import bs4
import pandas
import re
import time

chk = False

In [17]:
def getKeyword() :
    keyword = input('검색어를 입력해 주세요 : ')
    return keyword

In [18]:
def connectionSite(keyword, start):
    # 요청할 페이지의 주소
    site = 'https://openapi.naver.com/v1/search/blog.xml'
    # 파라미터 데이터
    params = {
        'query' : keyword,
        'start' : start,
        'display' : 100,
        'sort' : 'date'
    }
    # 헤더정보
    headers = {
        'X-Naver-Client-Id' : 'Fx7kgsRNPQ7XGU5LeQSE',
        'X-Naver-Client-Secret' : '2s01pestJ4'
    }
    # 요청한다.
    response = requests.get(site, params=params, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # print(soup.prettify())
    return soup

In [19]:
# 태그의 문자열에서 html 태그를 제거하고 반환하는 함수
def getText(tag, tag_name) :
    temp_str = tag.select(tag_name)[0].text.strip()
    temp_str = re.sub('<.+?>', '', temp_str, 0).strip()
    return temp_str


def getData(soup, keyword, start) :
    # item 태그들을 가져온다.
    item_list = soup.select('item')
    # print(item_list)
    # item 태그의 개수만큼 반복
    for item_tag in item_list :
        # title 값 추출
        title_str = getText(item_tag, 'title')
        # print(title_str)
        # link 추출
        link_str = list(item_tag.children)[2].strip()
        # print(link_str)
        # description 추출
        description_str = getText(item_tag, 'description')
        # print(description_str)
        bloggername_str = getText(item_tag, 'bloggername')
        bloggerlink_str = getText(item_tag, 'bloggerlink')
        postdate_str = getText(item_tag, 'postdate')

        data_dic = {
            'title' : [title_str],
            'link' : [link_str],
            'description' : [description_str],
            'bloggername' : [bloggername_str],
            'bloggerlink' : [bloggerlink_str],
            'postdate' : [postdate_str]
        }

        df = pandas.DataFrame(data_dic)

        global chk
        
        if chk == False :
            chk = True
            df.to_csv(f'{keyword}.csv', index=False, encoding='utf-8-sig')
        else :
            df.to_csv(f'{keyword}.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

        # total 값을 가져온다.
        total_str = soup.select('total')[0].text.strip()
        
    return int(total_str)
        
    

In [21]:
start = 1

keyword = getKeyword()

while True :
    time.sleep(0.5)
    soup = connectionSite(keyword, start)
    total = getData(soup, keyword, start)
    
    print(f'{start} / {total}')
    
    # 더이상 있는지 확인한다.
    start = start + 100

    # 1000개 이하일 경우
    if start >= total :
        break
    # 901번까지 왔다면 1000으로 셋팅한다.(마지막 페이지)
    elif start == 1001 :
        start = 1000
    # 1000번까지 왔다면 종료
    elif start == 1100 :
        break
    
    
    
    

검색어를 입력해 주세요 : 멀티캠퍼스
1 / 45972
101 / 45972
201 / 45972
301 / 45972
401 / 45972
501 / 45972
601 / 45972
701 / 45972
801 / 45972
901 / 45972
1000 / 45972
